In [1]:
# Importing the necessacory libs
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report,f1_score, accuracy_score, recall_score, precision_score
from nltk.stem import WordNetLemmatizer

In [2]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\subha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [76]:
corpus_df = pd.read_csv("blogtext.csv")
corpus_df.head(10)

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...
5,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",I had an interesting conversation...
6,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",Somehow Coca-Cola has a way of su...
7,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004","If anything, Korea is a country o..."
8,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",Take a read of this news article ...
9,3581210,male,33,InvestmentBanking,Aquarius,"09,June,2004",I surf the English news sites a l...


In [150]:
corpus_df.shape

(681284, 7)

In [77]:
# Taking some rows to initial pre processing & training
corpus_df_subset = corpus_df[:5000]
print(corpus_df_subset.shape)
corpus_df_subset["text"][0]

(5000, 7)


'           Info has been found (+/- 100 pages, and 4.5 MB of .pdf files) Now i have to wait untill our team leader has processed it and learns html.         '

In [78]:
import re
from nltk.tokenize.toktok import ToktokTokenizer
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
tokenizer = ToktokTokenizer()
stop_words  = set(stopwords.words('english'))

def remove_stopwords(text):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    filtered_tokens = [token for token in tokens if token not in stop_words]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

def text_prepare(text): ### The function will take in text and lower case it remove the stopwords, symbols and return it.
    text = text.lower()             ### Write a code which can change the input text to lowercase.
    text = re.sub(REPLACE_BY_SPACE_RE,'',text)            ### Write a code which replaces REPLACE_BY_SPACE_RE (above mentioned) symbols by space in text
    text = re.sub(BAD_SYMBOLS_RE,'',text)            ### Write a code which deletes symbols which are in BAD_SYMBOLS_RE (above mentioned) from text
    text = remove_stopwords(text)             ### Write a code which deletes stopwords from text
    return text

In [79]:
corpus_df_subset['text'] = [text_prepare(title) for title in corpus_df_subset['text']]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [80]:
corpus_df_subset['text'][0]

'info found + 100 pages 45 mb pdf files wait untill team leader processed learns html'

In [83]:
#name of available columns 
corpus_df_subset.head()

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004",info found + 100 pages 45 mb pdf files wait un...
1,2059027,male,15,Student,Leo,"13,May,2004",team members drewes van der laag urllink mail ...
2,2059027,male,15,Student,Leo,"12,May,2004",het kader van kernfusie op aarde maak je eigen...
3,2059027,male,15,Student,Leo,"12,May,2004",testing testing
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",thanks yahoos toolbar capture urls popupswhich...


In [88]:
corpus_df_subset.columns

Index(['id', 'gender', 'age', 'topic', 'sign', 'date', 'text'], dtype='object')

In [89]:
corpus_df_subset.dtypes

id         int64
gender    object
age        int64
topic     object
sign      object
date      object
text      object
dtype: object

In [90]:
corpus_df_subset['age'] = corpus_df_subset['age'].astype(str)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


### Label columns to merge: “gender”, “age”, “topic”, “sign”

In [91]:
corpus_df_subset['labels'] = corpus_df_subset[['gender','age','topic','sign']].apply(lambda x: ','.join(x), axis = 1)
corpus_df_subset_final = corpus_df_subset.drop(labels = ['date','gender', 'age','topic','sign','id'], axis = 1)
corpus_df_subset_final.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,text,labels
0,info found + 100 pages 45 mb pdf files wait un...,"male,15,Student,Leo"
1,team members drewes van der laag urllink mail ...,"male,15,Student,Leo"
2,het kader van kernfusie op aarde maak je eigen...,"male,15,Student,Leo"
3,testing testing,"male,15,Student,Leo"
4,thanks yahoos toolbar capture urls popupswhich...,"male,33,InvestmentBanking,Aquarius"


In [92]:
corpus_df_subset_final.shape

(5000, 2)

In [109]:
x_feature = corpus_df_subset_final['text']
corpus_df_subset_final['labels'] = corpus_df_subset_final['labels'].str.lower()
y_labels = corpus_df_subset_final['labels']
X_train, X_test, Y_train, Y_test = train_test_split(x_feature,y_labels, test_size = 0.3, random_state = 10)
Y_train.shape

(3500,)

### Vectorize the features (5 points)

In [110]:
# Create a Bag of Words using count vectorizer
vectorizer = CountVectorizer(min_df = 2,ngram_range = (1,2),stop_words = "english")
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)
print("X_train shape & sample",X_train.shape)
X_train[0]

X_train shape & sample (3500, 27292)


<1x27292 sparse matrix of type '<class 'numpy.int64'>'
	with 41 stored elements in Compressed Sparse Row format>

### Create a dictionary to get the count of every label i.e. the key will be label name and value will  be the total count of the label.

In [111]:
vectorizer_labels = CountVectorizer(min_df = 1,ngram_range = (1,1),stop_words = "english")
labels_vector = vectorizer_labels.fit_transform(y_labels)
vectorizer_labels.vocabulary_

{'male': 42,
 '15': 1,
 'student': 54,
 'leo': 39,
 '33': 9,
 'investmentbanking': 37,
 'aquarius': 21,
 'female': 33,
 '14': 0,
 'indunk': 35,
 'aries': 22,
 '25': 6,
 'capricorn': 28,
 '17': 3,
 'gemini': 34,
 '23': 4,
 'non': 45,
 'profit': 47,
 'cancer': 27,
 'banking': 25,
 '37': 13,
 'sagittarius': 50,
 '26': 7,
 '24': 5,
 'scorpio': 52,
 '27': 8,
 'education': 31,
 '45': 18,
 'engineering': 32,
 'libra': 40,
 'science': 51,
 '34': 10,
 '41': 15,
 'communications': 29,
 'media': 43,
 'businessservices': 26,
 'sports': 53,
 'recreation': 48,
 'virgo': 57,
 'taurus': 55,
 'arts': 23,
 'pisces': 46,
 '44': 17,
 '16': 2,
 'internet': 36,
 'museums': 44,
 'libraries': 41,
 'accounting': 20,
 '39': 14,
 '35': 11,
 'technology': 56,
 '36': 12,
 'law': 38,
 '46': 19,
 'consulting': 30,
 'automotive': 24,
 '42': 16,
 'religion': 49}

### Transform the labels

In [105]:
# initialising multilabelbinariser with all unique possible classes
mlb = MultiLabelBinarizer(classes=sorted(vectorizer_labels.vocabulary_.keys()))

In [106]:
y_labels[0]

'male,15,student,leo'

In [115]:
Y_train_labels = [["".join(re.findall("\w",f)) for f in lst] for lst in [s.split(",") for s in Y_train]]
Y_train_labels[0]

['male', '35', 'technology', 'aries']

In [113]:
y_train_transformed = mlb.fit_transform(Y_train_labels) # transforming entire set of lables

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:935: UserWarning: unknown class(es) ['communicationsmedia', 'museumslibraries', 'nonprofit', 'sportsrecreation'] will be ignored
  .format(sorted(unknown, key=str)))


In [116]:
y_train_transformed[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0])

In [117]:
# Converting entire se of labels into format required by mlb
y_test_labels = [["".join(re.findall("\w",f)) for f in lst] for lst in [s.split(",") for s in Y_test]]
y_test_labels[0]

['male', '15', 'student', 'aquarius']

In [139]:
y_test_transformed = mlb.fit_transform(y_test_labels) # transforming entire set of lables

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:935: UserWarning: unknown class(es) ['communicationsmedia', 'museumslibraries', 'nonprofit', 'sportsrecreation'] will be ignored
  .format(sorted(unknown, key=str)))


In [128]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(solver = 'lbfgs',max_iter = 1000)  # initiating the classifier
clf = OneVsRestClassifier(clf)

In [130]:
clf.fit(X_train,y_train_transformed) # Fitting on  train data

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 29 is present in all training examples.
  str(classes[c]))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 41 is present in all training examples.
  str(classes[c]))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 43 is present in all training examples.
  str(classes[c]))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 44 is present in all training examples.
  str(classes[c]))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 45 is present in all training examples.
  str(classes[c]))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 47 is present in all training examples.
  str(classes[c]))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\multiclass.py:76: UserWarning: Label not 48

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False),
          n_jobs=None)

In [131]:
Y_pred = clf.predict(X_test)

In [132]:
Y_pred[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [135]:
y_train_transformed[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0])

In [137]:
print("Train Accuracy:",clf.score(X_train,y_train_transformed))

Train Accuracy: 0.9522857142857143


In [140]:
print("Test Accuracy:" + str(accuracy_score(y_test_transformed, Y_pred)))
print("F1: " + str(f1_score(y_test_transformed, Y_pred, average='micro')))
print("F1_macro: " + str(f1_score(y_test_transformed, Y_pred, average='macro')))
print("Precision: " + str(precision_score(y_test_transformed, Y_pred, average='micro')))
print("Recall: " + str(recall_score(y_test_transformed, Y_pred, average='micro')))

Test Accuracy:0.5193333333333333
F1: 0.7333516031789531
F1_macro: 0.25809766262653944
Precision: 0.8029605921184236
Recall: 0.6748486886348353


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


###  Print true label and predicted label for any five examples

In [148]:
y_pred_inverse = mlb.inverse_transform(Y_pred)   # inverse transforming predited label data
y_test_transformed_inverse =  mlb.inverse_transform(y_test_transformed) 

In [149]:
for i in range(6):
    print("Example  :predicted label: ",y_pred_inverse[i])
    print("Example  true label: ",y_test_transformed_inverse[i])

Example  :predicted label:  ('aquarius', 'female', 'indunk')
Example  true label:  ('15', 'aquarius', 'male', 'student')
Example  :predicted label:  ('34', 'female', 'indunk', 'sagittarius')
Example  true label:  ('34', 'female', 'indunk', 'sagittarius')
Example  :predicted label:  ('34', 'female', 'indunk', 'sagittarius')
Example  true label:  ('34', 'female', 'indunk', 'sagittarius')
Example  :predicted label:  ('35', 'aries', 'male', 'technology')
Example  true label:  ('35', 'aries', 'male', 'technology')
Example  :predicted label:  ('35', 'aries', 'male', 'technology')
Example  true label:  ('35', 'aries', 'male', 'technology')
Example  :predicted label:  ('female',)
Example  true label:  ('23', 'indunk', 'male', 'sagittarius')
